<a href="https://colab.research.google.com/github/mbaliu-treino/Desenvolve/blob/main/LEARN_C_DS_Sistema_de_Recomendacao_KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <font color=orange><b>Machine Learning: intro a sistemas de recomendação em Python</b></font>

<font color=gray size=2><a href="https://colab.research.google.com/drive/1GmuhJfaGYBLscfKt5arSnNWBJ4U63vgJ">Arquivo Colab</a></font>
<!--- Para indicar o arquivo a ser trabalhado a partir do Github, sem ter que criar uma nova cópias --->


<ul><font size=2 color=gray><b>FICHA TÉCNICA</b>
<li><a href="https://cursos.alura.com.br/course/introducao-a-sistemas-de-recomendacao-com-python"><font size=2 color=gray>Machine Learning: intro a sistemas de recomendação em Python</a>
<li>Carga Horária: 8 h
<li>Instrutora: Guilherme Silveira
<li>Data de Início: 10-2022
</ul>


<hr color=gray><br>

<!-- DESCRIÇÃO DOS TÓPICOS APRENDIDOS -->
<h3><b>Conteúdo / Aprendizagem:</b></h3>

- Pensamento de heurísticas e algoritmos
    - Sistemas baseados em popularidade e melhor nota,
    - Adição de filtros
    - Desvantangens nestes tipos de recomendações
- Recomendações baseadas em similaridade de usuários
- Recomendação baseado em modelos de itens e usuários
- Sistemas baseados em filtros colaborativos ou por conteúdo
- Desafios de recomendação
- Implementação de K-Nearest-Neighbours do zero

* Utilizar o for de maneira mais curta;
Fazer uma função para ver quais filmes estão mais próximos da escolha das pessoas.
Testar nosso sistema;
Filtrar as distâncias;
Analisar um subconjunto;
Utilizar a função iloc para acessar as linhas;
Ordenar pela nota de forma crescente com o parâmetro ascending=False;
Extrair os usuários mais próximos;
Localizar baseado no índice com set_index(‘usuarioid).loc[usuarios similares];
O que é o KNN.

# <font color=orange>RESUMO</font>
<!-- DESCRIÇÃO DO PROCESSO DE PROCESAMENTO -->

<a href=https://docs.google.com/spreadsheets/d/1tTygYlq9r7nkUsw9a25N5_z57-de_59tSBCIVPw6KUw><font size=2 color=gray>ROTEIROS Data Science</font></a><!-- Planilha de roteiros para Data Science -->

* <font color=orange><b>



# <font color=orange>**CURSO**</font>
<!-- Descrição do projeto a ser realizado -->

**Projeto**: o projeto aqui desenvolvido é de um **sistrema de recomendação** baseado em similaridade de usuários através da análise de itens dos usuários. O case é de recomedação de filmes.


<!-- Material final oferecido no curso -->
<a href="https://github.com/alura-cursos/introducao-a-sistemas-de-recomendacao-com-python/tree/aula06"><font size=2; color=gray>Material de referencial do curso</a></font>


## 0 - Ambiente

Podemos criar ambientes virtuais para que os projetos não sejam impactados pela quantidade de bibliotecas instaladas.

In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

print('numpy:', np.__version__)
print('pandas:', pd.__version__)

numpy: 1.21.6
pandas: 1.3.5


## 1 - Dados

Os dados usuados para o sistema de recomendação é da base de dados MovieLens, que possui duas bases de dados:

1. PEQUENO: 
    - 100.000 registros de avaliações
    - 3.600 tags aplciados
    - 9.000 filmes
    - 600 usuários
    - 1 MB
2. COMPLETO:
    - 27.000.000 registros de avaliações
    - 1.100.000 tags aplciados
    - 58.000 filmes
    - 280.000 usuários
    - 265 MB

(Para a base completa, podemos usar o Spark para avaliar se há ganho de performance.)

In [2]:
url_movielens_small = 'https://files.grouplens.org/datasets/movielens/ml-latest-small.zip'
url_movielens_full = 'https://files.grouplens.org/datasets/movielens/ml-latest.zip'


dataset_size = ''  # small, ''

if dataset_size:
    !wget $url_movielens_small
    !unzip "/content/ml-latest-small.zip"
else:
    !wget $url_movielens_full
    !unzip "/content/ml-latest.zip"

--2022-10-31 17:05:22--  https://files.grouplens.org/datasets/movielens/ml-latest.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 277113433 (264M) [application/zip]
Saving to: ‘ml-latest.zip’

ml-latest.zip       100%[===================>] 264.28M  76.5MB/s    in 3.7s    

2022-10-31 17:05:26 (72.1 MB/s) - ‘ml-latest.zip’ saved [277113433/277113433]

Archive:  /content/ml-latest.zip
   creating: ml-latest/
  inflating: ml-latest/links.csv     
  inflating: ml-latest/tags.csv      
  inflating: ml-latest/genome-tags.csv  
  inflating: ml-latest/ratings.csv   
  inflating: ml-latest/README.txt    
  inflating: ml-latest/genome-scores.csv  
  inflating: ml-latest/movies.csv    


In [3]:
# Filmes
filmes = pd.read_csv('/content/ml-latest/movies.csv')
filmes.columns = ['filmeId', 'titulo', 'generos']
filmes = filmes.set_index('filmeId')
filmes.head()

,titulo,generos
filmeId,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy


In [4]:
# Notas
notas = pd.read_csv('/content/ml-latest/ratings.csv')
notas.columns = ['usuarioId', 'filmeId', 'nota', 'momento']
notas.head()

,usuarioId,filmeId,nota,momento
0,1,307,3.5,1256677221
1,1,481,3.5,1256677456
2,1,1091,1.5,1256677471
3,1,1257,4.5,1256677460
4,1,1449,4.5,1256677264


In [5]:
notas.describe().round(1)

,usuarioId,filmeId,nota,momento
count,27753444.0,27753444.0,27753444.0,2.775344e+07
mean,141942.0,18488.0,3.5,1.193122e+09
std,81707.4,35102.6,1.1,2.160482e+08
min,1.0,1.0,0.5,7.896520e+08
25%,71176.0,1097.0,3.0,9.986053e+08
50%,142022.0,2716.0,3.5,1.174256e+09
75%,212459.0,7150.0,4.0,1.422744e+09
max,283228.0,193886.0,5.0,1.537945e+09


## 2 - Heuristica da Recomedação e o princípio da colaboração

Quando se deseja recomendar algo, surge a pergunta: Como podemos recomendar algo? 

Ou seja, tendo vários filmes e várias notas, qual filme que recomendaremos? Logo, temos pessoas que assitiram filmes e deram notas. Cada filme teve uma quantidade diferente de visualizações. As notas representam os votos das pessoas.

(1) Neste problema, temos primeiro uma questão de AMOSTRAGEM. Nela assumiremos a premissa que estas notas são representações razoáveis do mundo real. Ou seja, que esta **amostra é representativa**.

(2) Outro aspecto da recomendação é qual tipo de informação é possuímos para realizar a recomendação. Usando somente os registros de notas, não temos conhecimento dos usuários, mas temos a popularidade do filme e o que as pessoas em geral acharam dele. Ou seja, os outros usuários da plataforma colaboraram votando nos filmes.

- Popularidade;
- Melhor nota


In [6]:
# Mais votados
"""Tendo dois conjuntos indexados é possível usar a atribuição 
para realizar o join. O Pandas irá usar o index como chave de conexão"""

mais_votados = notas['filmeId'].value_counts()
filmes['total_votos'] = mais_votados
filmes

,titulo,generos,total_votos
filmeId,,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,68469.0
2,Jumanji (1995),Adventure|Children|Fantasy,27143.0
3,Grumpier Old Men (1995),Comedy|Romance,15585.0
4,Waiting to Exhale (1995),Comedy|Drama|Romance,2989.0
5,Father of the Bride Part II (1995),Comedy,15474.0
...,...,...,...
193876,The Great Glinka (1946),(no genres listed),1.0
193878,Les tribulations d'une caissière (2011),Comedy,1.0
193880,Her Name Was Mumu (2016),Drama,1.0


In [7]:
# 5 filmes mais votados
filmes.sort_values('total_votos', ascending=False).head(10)

,titulo,generos,total_votos
filmeId,,,
318,"Shawshank Redemption, The (1994)",Crime|Drama,97999.0
356,Forrest Gump (1994),Comedy|Drama|Romance|War,97040.0
296,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,92406.0
593,"Silence of the Lambs, The (1991)",Crime|Horror|Thriller,87899.0
2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller,84545.0
260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi,81815.0
480,Jurassic Park (1993),Action|Adventure|Sci-Fi|Thriller,76451.0
527,Schindler's List (1993),Drama|War,71516.0
110,Braveheart (1995),Action|Drama|War,68803.0


In [8]:
# + nota média dos filmes
nota_media = notas.groupby('filmeId')['nota'].mean()
filmes['nota_media'] = nota_media

filmes.sort_values('total_votos', ascending=False).head(10)

,titulo,generos,total_votos,nota_media
filmeId,,,,
318,"Shawshank Redemption, The (1994)",Crime|Drama,97999.0,4.424188
356,Forrest Gump (1994),Comedy|Drama|Romance|War,97040.0,4.056585
296,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,92406.0,4.173971
593,"Silence of the Lambs, The (1991)",Crime|Horror|Thriller,87899.0,4.151412
2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller,84545.0,4.149695
260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi,81815.0,4.120455
480,Jurassic Park (1993),Action|Adventure|Sci-Fi|Thriller,76451.0,3.665034
527,Schindler's List (1993),Drama|War,71516.0,4.257502
110,Braveheart (1995),Action|Drama|War,68803.0,4.008481


In [9]:
# 10 filmes com maior nota
filmes.sort_values('nota_media', ascending=False).head(10)

,titulo,generos,total_votos,nota_media
filmeId,,,,
169338,Brad Williams: Daddy Issues (2016),Comedy,2.0,5.0
187729,Ab-normal Beauty (2004),Horror,1.0,5.0
172149,Back to You and Me (2005),Drama|Romance,1.0,5.0
160966,You're Human Like the Rest of Them (1967),(no genres listed),1.0,5.0
134387,At Ellen’s Age (2011),Comedy|Drama,1.0,5.0
98437,Bed of Roses (1933),Comedy|Drama|Romance,1.0,5.0
134433,Le nuove comiche (1994),(no genres listed),1.0,5.0
172151,The Shocking Miss Pilgrim (1947),Comedy|Romance,1.0,5.0
134605,Men Don't Cry (1968),Comedy,1.0,5.0


**Dificuldades da heurística univariada**: A recomendação baseado em somente uma única coluna pode apresentar diversas falhas substanciais no objetivo da recomendação. Na seleção dos filmes mais populares, primeiro não definimos o que seria a popularidade e, segundo, alguns filmes com notas mais baixas foram selecionados, já que diversas pessoas votaram nele, mas não necessariamente acharam ele bom.

O segundo experimento foi usado a nota como elemento para a seleção. No entanto, percebemos que filmes com poucas notas (todos só tinham um voto) foram os que tiveram as melhores notas, já que possuem alto grau de viés. Neste tipo de seleção encontramos o problema dos nichos, filmes que são assistidos por poucas pessoas. Neste caso, os filmes provavelmente não irão interessas à maioria das pessoas.

Com isso, encontramos um dilema da heurística da recomendação. O simples rankeamento não é suficiente. O que podemos fazer é realizar filtros, usar *threshold*, para limitar algumas situações. 

Mas nestes casos teremos que tomar uma decisão arbitrária, para obter resultados melhores. Podemos ajustar estes filtros com a reação dos clientes. Se for satisfatório, o sistema de recomendação estará execendo a função dele, mas se os clientes não estiverem gostando do sistema, deveremos alterar os filtros.

In [10]:
# Filtro 
seletor = filmes['total_votos'] > 50
filmes[seletor].sort_values('nota_media', ascending=False).head(10)

,titulo,generos,total_votos,nota_media
filmeId,,,,
171011,Planet Earth II (2016),Documentary,853.0,4.486518
159817,Planet Earth (2006),Documentary,1384.0,4.458092
318,"Shawshank Redemption, The (1994)",Crime|Drama,97999.0,4.424188
170705,Band of Brothers (2001),Action|Drama|War,984.0,4.399898
174053,Black Mirror: White Christmas (2014),Drama|Horror|Mystery|Sci-Fi|Thriller,1074.0,4.350559
171495,Cosmos,(no genres listed),157.0,4.343949
172591,The Godfather Trilogy: 1972-1990 (1992),(no genres listed),421.0,4.339667
858,"Godfather, The (1972)",Crime|Drama,60904.0,4.332893
50,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,62180.0,4.291959


## 3 - Heurística através do Usuário

Para melhorar a qualidade das recomendações precisamos é ter mais critérios para a seleção. Uma forma trazer novos critérios para a realziação de recomendações é acrescentando informações dos **usuários** para o sistema.

Com isso, pode-se analisar o histórico de filmes assistidos pelo usuário e aplicar no sistema de recomendação. Com isso, tem-se uma recomendação mais personalizada.

Mas algumas dilemas ainda persistem, tais como se filmes de nicho devem ou não ser mostrados e como identificá-los.


In [11]:
# Exemplo de situação
eu_assisti = [1, 21, 19, 10, 11, 7, 2]
filmes.loc[eu_assisti]

,titulo,generos,total_votos,nota_media
filmeId,,,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,68469.0,3.886649
21,Get Shorty (1995),Comedy|Crime|Thriller,25699.0,3.568368
19,Ace Ventura: When Nature Calls (1995),Comedy,24913.0,2.642014
10,GoldenEye (1995),Action|Adventure|Thriller,33086.0,3.431633
11,"American President, The (1995)",Comedy|Drama|Romance,19669.0,3.660278
7,Sabrina (1995),Comedy|Romance,15301.0,3.371348
2,Jumanji (1995),Adventure|Children|Fantasy,27143.0,3.246583


O último filme visto era do gênero aventura, infantil e fantasia. Qual outro filme semelhante que pode ser recomendado?

In [12]:
# Seleção de filmes bem votados do mesmo genero
df = filmes.query('generos == "Adventure|Children|Fantasy"')
df = df.sort_values('nota_media', ascending=False)
df.head()

,titulo,generos,total_votos,nota_media
filmeId,,,,
189473,"Red, Honest, in Love (1984)",Adventure|Children|Fantasy,2.0,4.250000
191773,Up in the Sky (2016),Adventure|Children|Fantasy,2.0,4.000000
169092,The Story of the Voyages (1983),Adventure|Children|Fantasy,10.0,4.000000
177369,Three Fat Men (1966),Adventure|Children|Fantasy,6.0,3.833333
167132,Perri (1957),Adventure|Children|Fantasy,3.0,3.833333


In [13]:
# Seleção de filmes bem votados do mesmo genero
"""Pode-se realizar a mesma operação heurística para remoção de filmes de nicho
O resultado traz alguns filmes de nicho e outros populares"""

df = filmes.query('total_votos > 50')
df = df.query('generos == "Adventure|Children|Fantasy"')
df = df.sort_values('nota_media', ascending=False)
# Remoção dos filmes já vistos
df = df.drop(eu_assisti, errors='ignore')
df.head()

,titulo,generos,total_votos,nota_media
filmeId,,,,
4896,Harry Potter and the Sorcerer's Stone (a.k.a. ...,Adventure|Children|Fantasy,27434.0,3.674892
80748,Alice in Wonderland (1933),Adventure|Children|Fantasy,117.0,3.564103
2161,"NeverEnding Story, The (1984)",Adventure|Children|Fantasy,12426.0,3.500161
50601,Bridge to Terabithia (2007),Adventure|Children|Fantasy,1987.0,3.430549
41566,"Chronicles of Narnia: The Lion, the Witch and ...",Adventure|Children|Fantasy,13042.0,3.409216


In [14]:
# Seleção de filmes bem votados do mesmo genero
"""Pode-se realizar a mesma operação heurística para remoção de filmes de nicho,
mas baseado no nível de nicho (popularidade) do filme assistido"""

pop_filme = 27143  # popularidade do filme Jumanji 
pop_filme *= 0.25

df = filmes.query(f'total_votos > {pop_filme}')
df = df.query('generos == "Adventure|Children|Fantasy"')
df = df.sort_values('nota_media', ascending=False)
df = df.drop(eu_assisti, errors='ignore')
df.head()

,titulo,generos,total_votos,nota_media
filmeId,,,,
4896,Harry Potter and the Sorcerer's Stone (a.k.a. ...,Adventure|Children|Fantasy,27434.0,3.674892
2161,"NeverEnding Story, The (1984)",Adventure|Children|Fantasy,12426.0,3.500161
41566,"Chronicles of Narnia: The Lion, the Witch and ...",Adventure|Children|Fantasy,13042.0,3.409216
60,"Indian in the Cupboard, The (1995)",Adventure|Children|Fantasy,7601.0,3.071241


## 4 - Modelo baseado em distância entre usuários

Existem várias possibilidades para o <font color=orange><b>problema de recomendação</b></font>, como sendo um problema de regressão (com as probabilidades), de classificação (qual elemento a recomendar), de ranking (onde está o filme a ser recomendado).

O novo modelo aqui criado usa a nota do usuário ao filmes para identificar o seu <font color=orange><b>PERFIL</b></font>. 

O conceito de PERFIL já foi usado anteriormente quando considerado o último filme assistido pelo usuário. Isso já é um modelo simples do usuário.

Nesse novo modelo de perfil é considerado a nota para identificar outros <font color=orange><b>usuários similares</b></font>.

<font color=orange><b>DISTÂNCIA DE SIMILARIDADE</b></font>: Uma forma de calcular a similaridade entre duas entidades, neste caso usuários, pode-se calcular a distância entre eles. Usando os atributos numéricos dos usuários é possível alguma medida de distância entre eles, que representaria a distância entre seus perfis. 

Um algorítmo que utiliza este mesmo princípio é o KNN.

O novo modelo de recomendação irá utilizar utilizar o princípio da **sistema colaborativo**, em que os próprios usuários atribuem notas para os filmes. Com isso será possível criar um perfil de cada usuário para identificar outros usuários similares e conseguir recomendar filmes que eles gostaram.

### 4.1 - Distância entre dois usuários

Através da comparação entre os mesmos filmes votados em uma dupla de usuários é obtida a distância euclidiana entre eles. Isto poderá ser **abstraído** e **generalizado** para ser usada como uma medida da distância entre o gosto desses dois usuários.

In [15]:
# Método de pitágoras no numpy
np.linalg.norm([3,4])

5.0

In [16]:
# Função de distância
def distancia_vetores(a, b):
    return np.linalg.norm(a-b)

In [17]:
# Usuário 1: notas
notas_do_usuario = notas.query('usuarioId == 1')[['filmeId', 'nota']].set_index('filmeId')
notas_do_usuario

,nota
filmeId,
307,3.5
481,3.5
1091,1.5
1257,4.5
1449,4.5
1590,2.5
1591,1.5
2134,4.5
2478,4.0


In [18]:
# Função de seleção das notas de usuário
def notas_do_usuario(usuario):
    notas_usuario = notas.query(f'usuarioId == {usuario}')[['filmeId', 'nota']].set_index('filmeId')
    return notas_usuario

In [19]:
notas_do_usuario(1)

,nota
filmeId,
307,3.5
481,3.5
1091,1.5
1257,4.5
1449,4.5
1590,2.5
1591,1.5
2134,4.5
2478,4.0


In [20]:
user1 = notas_do_usuario(1)
user4 = notas_do_usuario(4)

# LEFT JOIN
# diferenca = user1.join(user4, lsuffix='_l', rsuffix='_r').dropna()

diferenca = pd.merge(user1, user4, 
         left_index=True, right_index=True, how='inner')

diferenca

,nota_x,nota_y
filmeId,,
481,3.5,4.0
1091,1.5,1.0
1590,2.5,3.5
1591,1.5,3.5
2478,4.0,3.0
2840,3.0,3.5
2986,2.5,1.5
3020,4.0,3.5
3698,3.5,4.0


In [21]:
distancia_vetores(diferenca['nota_x'], diferenca['nota_y'])

3.0413812651491097

In [22]:
def distancia_usuarios(usuario_id1, usuario_id2):
    notas1 = notas_do_usuario(usuario_id1)
    notas2 = notas_do_usuario(usuario_id2)
    diferenca = notas1.join(notas2, lsuffix='_l', rsuffix='_r').dropna()
    distancia = distancia_vetores(diferenca['nota_l'], diferenca['nota_r'])
    return [usuario_id1, usuario_id2, distancia]

distancia_usuarios(1,4)

[1, 4, 3.0413812651491097]

### 4.2 - Distância entre um e todos

Anteriormente calculamos a distância entre dois usuários. Agora é preciso saber a distância de um usuário com todos os outros, para saber com quais ele é mais semelhante e a quais ele é mais distante.

In [23]:
# Conjunto de usuários
quantidade_usuarios = notas['usuarioId'].unique()
print('Total de usuários no sistema:', len(quantidade_usuarios))

Total de usuários no sistema: 283228


In [24]:
# Distância entre um usuário e todos os outros [forma imperativa]

def distancia_de_todos(usuario_origem):
    distancias = []
    for usuario_id in quantidade_usuarios[:25]:
        informacoes = distancia_usuarios(usuario_origem, usuario_id)
        distancias.append(informacoes)

    return distancias

distancia_de_todos(1)

[[1, 1, 0.0],
 [1, 2, 0.0],
 [1, 3, 0.0],
 [1, 4, 3.0413812651491097],
 [1, 5, 0.0],
 [1, 6, 0.5],
 [1, 7, 0.0],
 [1, 8, 0.0],
 [1, 9, 0.0],
 [1, 10, 0.0],
 [1, 11, 0.0],
 [1, 12, 0.0],
 [1, 13, 0.0],
 [1, 14, 0.0],
 [1, 15, 0.0],
 [1, 16, 0.0],
 [1, 17, 0.0],
 [1, 18, 0.0],
 [1, 19, 0.5],
 [1, 20, 0.0],
 [1, 21, 0.0],
 [1, 22, 0.0],
 [1, 23, 0.0],
 [1, 24, 0.0],
 [1, 25, 0.0]]

In [25]:
# Distância entre um usuário e todos os outros [list comprehensive]
def distancia_de_todos(usuario_origem):
    quantidade_usuarios = notas['usuarioId'].unique()
    todos_usuarios = quantidade_usuarios[:300]
    distancias = (distancia_usuarios(1, usuario_id) for usuario_id in todos_usuarios)
    distancias = pd.DataFrame(distancias, columns=['origem', 'outra_pessoa', 'distancia'])
    return distancias

distancia_de_todos(1)

,origem,outra_pessoa,distancia
0,1,1,0.000000
1,1,2,0.000000
2,1,3,0.000000
3,1,4,3.041381
4,1,5,0.000000
...,...,...,...
295,1,296,0.000000
296,1,297,0.000000
297,1,298,0.500000
298,1,299,0.000000


Ao avaliar este resultado, percebe-se que há mutas notas 0. Provavelmente são eventos em que não há filmes assistido entre os dois.

In [26]:
# Teste de comparação entre notas dadas por dois usuários
notas_do_usuario(1).join(notas_do_usuario(5), lsuffix='_1').dropna()

,nota_1,nota
filmeId,,


### 4.3 - Ajuste da função de distância

Devido aos casos de poucas notas ou até mesmo nenhuma nota prejudicar o conceito de similaridade, iremos ajustar a função da distância para limitar a quantidade mpinima de filmes avaliada por dosi usuários.

In [27]:
def distancia_usuarios(usuario_id1, usuario_id2):
    notas1 = notas_do_usuario(usuario_id1)
    notas2 = notas_do_usuario(usuario_id2)
    diferencas = notas1.join(notas2, lsuffix='_l', rsuffix='_r').dropna()
    minimo = 5
    if len(diferencas) < minimo:
        return [usuario_id1, usuario_id2, 100000]

    distancia = distancia_vetores(diferencas['nota_l'], diferencas['nota_r'])
    return [usuario_id1, usuario_id2, distancia]

distancia_usuarios(1,4)

[1, 4, 3.0413812651491097]

In [28]:
distancia_de_todos(1)

,origem,outra_pessoa,distancia
0,1,1,0.000000
1,1,2,100000.000000
2,1,3,100000.000000
3,1,4,3.041381
4,1,5,100000.000000
...,...,...,...
295,1,296,100000.000000
296,1,297,100000.000000
297,1,298,100000.000000
298,1,299,100000.000000


### 4.4 - Ordenando usuários por distância

In [29]:
def mais_proximos(usuario_origem):
    distancias = distancia_de_todos(usuario_origem)
    distancias = distancias.sort_values('distancia', ascending=True)
    distancias = distancias.set_index('outra_pessoa').drop(usuario_origem)
    return distancias

mais_proximos(1)

,origem,distancia
outra_pessoa,,
214,1,1.118034
235,1,1.732051
4,1,3.041381
26,1,3.640055
56,1,4.974937
...,...,...
100,1,100000.000000
99,1,100000.000000
98,1,100000.000000


### 4.5 - Refatoração para teste

O algorítmo não possui uma grande performance, então iremos limitar a quantidade de elementos comparados para acelerar as execuções de testes.

In [30]:
# Distância entre um usuário e todos os outros [list comprehensive]
def distancia_de_todos(usuario_origem, amostra=None):
    todos_usuarios = notas['usuarioId'].unique()
    if amostra:
        todos_usuarios = todos_usuarios[:amostra]
    distancias = (distancia_usuarios(1, usuario_id) for usuario_id in todos_usuarios)
    distancias = pd.DataFrame(distancias, columns=['origem', 'outra_pessoa', 'distancia'])
    return distancias

distancia_de_todos(1, amostra=300)

,origem,outra_pessoa,distancia
0,1,1,0.000000
1,1,2,100000.000000
2,1,3,100000.000000
3,1,4,3.041381
4,1,5,100000.000000
...,...,...,...
295,1,296,100000.000000
296,1,297,100000.000000
297,1,298,100000.000000
298,1,299,100000.000000


In [31]:
def mais_proximos(usuario_origem, amostra=None):
    distancias = distancia_de_todos(usuario_origem, amostra=amostra)
    distancias = distancias.sort_values('distancia', ascending=True)
    distancias = distancias.set_index('outra_pessoa').drop(usuario_origem)
    return distancias

mais_proximos(1, amostra=50)

,origem,distancia
outra_pessoa,,
4,1,3.041381
26,1,3.640055
48,1,100000.000000
28,1,100000.000000
29,1,100000.000000
30,1,100000.000000
31,1,100000.000000
32,1,100000.000000
33,1,100000.000000


### 4.6 - Limpeza das distâncias excessivas

Para remover os casos em que não se aplica, podemos removê-los.

In [32]:
def distancia_usuarios(usuario_id1, usuario_id2):
    notas1 = notas_do_usuario(usuario_id1)
    notas2 = notas_do_usuario(usuario_id2)
    diferencas = notas1.join(notas2, lsuffix='_l', rsuffix='_r').dropna()
    minimo = 5
    if len(diferencas) < minimo:
        return None

    distancia = distancia_vetores(diferencas['nota_l'], diferencas['nota_r'])
    return [usuario_id1, usuario_id2, distancia]

distancia_usuarios(1,4)

[1, 4, 3.0413812651491097]

A função `filter` retorna os dados que não forem compatíveis com o primeiro argumento de comparação. Outro aspecto importante é que ele não retona uma lista, mas sim um objeto filter, como um *generator*.

In [33]:
# Distância entre um usuário e todos os outros [list comprehensive]
def distancia_de_todos(usuario_origem, amostra=None):
    todos_usuarios = notas['usuarioId'].unique()
    if amostra:
        todos_usuarios = todos_usuarios[:amostra]
    distancias = (distancia_usuarios(1, usuario_id) for usuario_id in todos_usuarios)
    distancias = filter(None, distancias)
    distancias = pd.DataFrame(distancias, columns=['origem', 'outra_pessoa', 'distancia'])
    return distancias


mais_proximos(1, amostra=100)

,origem,distancia
outra_pessoa,,
4,1,3.041381
26,1,3.640055
56,1,4.974937


## 5 - Implementação da recomendação baseado no usuário

### 5.1 - Baseado em um único usuário

In [34]:
# Identificação do usuário mais similar
usuario_chave = 1
similares = mais_proximos(usuario_chave, amostra=50)

usuario_mais_similar = similares.iloc[0].name

In [35]:
def sugestao(usuario_chave, amostra=None):
    # Notas do usuário mais similar
    similares = mais_proximos(usuario_chave, amostra=amostra)
    usuario_mais_similar = similares.iloc[0].name
    notas_do_similar = notas_do_usuario(usuario_mais_similar)

    # Remoção dos filmes já assistidos
    filmes_ja_assitidos = notas_do_usuario(usuario_chave).index
    notas_do_similar = notas_do_similar.drop(filmes_ja_assitidos, errors='ignore')
    recomendacoes = notas_do_similar.sort_values('nota', ascending=False)
    return recomendacoes.head(10).join(filmes)

sugestao(1, amostra=50)

,nota,titulo,generos,total_votos,nota_media
filmeId,,,,,
2193,5.0,Willow (1988),Action|Adventure|Fantasy,10802.0,3.456814
27773,5.0,Old Boy (2003),Mystery|Thriller,12152.0,4.070935
349,5.0,Clear and Present Danger (1994),Action|Crime|Drama|Thriller,34547.0,3.667598
8132,5.0,Gladiator (1992),Action|Drama,3853.0,3.914093
8360,5.0,Shrek 2 (2004),Adventure|Animation|Children|Comedy|Musical|Ro...,21573.0,3.487554
8371,5.0,"Chronicles of Riddick, The (2004)",Action|Sci-Fi|Thriller,6032.0,3.325514
318,5.0,"Shawshank Redemption, The (1994)",Crime|Drama,97999.0,4.424188
8636,5.0,Spider-Man 2 (2004),Action|Adventure|Sci-Fi|IMAX,21989.0,3.453454
316,5.0,Stargate (1994),Action|Adventure|Sci-Fi,36996.0,3.370405


### 5.2 - Baseado em um conjunto de usuários (KNN)

Ao invés de se basear em um único elemento, neste caso, um usuário, podemos usar um conjunto de usuários mais similares para identificar qual o gosto genérico deles para recomendar a aum usuário. Então é a sugestão baseada em k indivíduos mais próximos, KNN.

Com isso, busca-se obter maior generalização e reduzir o impacto da idiossincrasia de poucos usuários.

In [38]:
# Adicição do número do tamanho do conjunto dos mais próximos
def mais_proximos(usuario_origem, k_mais_proximos=10, amostra=None):
    distancias = distancia_de_todos(usuario_origem, amostra=amostra)
    distancias = distancias.sort_values('distancia', ascending=True)
    distancias = distancias.set_index('outra_pessoa').drop(usuario_origem)
    return distancias.head(k_mais_proximos)

mais_proximos(1, amostra=50)

In [40]:
def sugestao_knn(usuario_chave, k_mais_proximos=10, amostra=None):
    # Notas do usuário mais similar
    similares = mais_proximos(usuario_chave, k_mais_proximos=k_mais_proximos, amostra=amostra)
    usuario_mais_similar = similares.index
    notas_dos_similares = notas.set_index('usuarioId').loc[usuario_mais_similar]
    recomendacoes = notas_dos_similares.groupby('filmeId')['nota'].mean()
    recomendacoes = recomendacoes.sort_values('nota', ascending=False)
    return recomendacoes.head(10).join(filmes)


usuario_chave=1
k_mais_proximos = 10

numero_de_usuarios_a_analisar = 50
sugestao_knn(usuario_chave, k_mais_proximos=k_mais_proximos, amostra=numero_de_usuarios_a_analisar)

Int64Index([4, 26], dtype='int64', name='outra_pessoa')

* Neste problema, o algorítmo não possui uma quantidade de colunas fixas, já que alguns filmes só foram assistidos por uma dupla, mas não por outra. Para resolver este problema, a distância é calculada somente entre dois usuários, usando as dimensões em comum entre eles.
* Cálculo da distância:

**REFINAMENTO**

Como encontrar um recomendador melhor?

* Refinar o cálculo de distância:
* Refinar a forma de agrupamento dos K usuários: grupos de usuários, grupos de filmes de acordo com o gênero, 